<a href="https://colab.research.google.com/github/Han1018/BeatGAN/blob/GenerateMidiAPI/Colab_API/MuseGan_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 導入雲端硬碟

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 下載必備套件

In [27]:
!pip install pypianoroll==0.4.6
!pip install tf-nightly
!pip install gast==0.2.0
!pip install flask-ngrok

  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Attempting uninstall: gast
    Found existing installation: gast 0.2.0
    Uninstalling gast-0.2.0:
      Successfully uninstalled gast-0.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 1.15.2 requires gast==0.2.2, but you have gast 0.4.0 which is incompatible.
kapre 0.3.5 requires tensorflow>=2.0.0, but you have tensorflow 1.15.2 which is incompatible.


  Using cached gast-0.2.0-py3-none-any.whl
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 1.15.2 requires gast==0.2.2, but you have gast 0.2.0 which is incompatible.
tf-nightly 2.8.0.dev20211001 requires gast==0.4.0, but you have gast 0.2.0 which is incompatible.
kapre 0.3.5 requires tensorflow>=2.0.0, but you have tensorflow 1.15.2 which is incompatible.


### 載入一代tf

In [28]:
%tensorflow_version 1.10
import tensorflow as tf
print(tf.__version__)

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.10`. This will be interpreted as: `1.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.
1.15.2


### 更改路徑

In [29]:
import os
os.chdir("/content/drive/MyDrive/museGan") #更改路徑
dir='/content/drive/MyDrive/museGan'
#os.getcwd() #查看當前路徑

In [30]:
"""This script performs inference from a trained model."""

import logging
import argparse
from pprint import pformat
import numpy as np
import scipy.stats
import tensorflow as tf
from config import LOGLEVEL, LOG_FORMAT
from data_predict_version import load_data, get_samples
from model import Model
from utils import make_sure_path_exists, load_yaml, update_not_none
LOGGER = logging.getLogger("musegan.inference")

In [31]:
def parse_arguments(folderName):

    #設定以哪個資料夾下的Model做預測
    # predict_folder='exp/default_2/'
    predict_folder = folderName

    """Parse and return the command line arguments."""
    parser = argparse.ArgumentParser()
    parser.add_argument('--result_dir',default=predict_folder,
                        help="Directory where the results are saved.")
    parser.add_argument('--checkpoint_dir',default=predict_folder+'model/',
                        help="Directory that contains checkpoints.")
    parser.add_argument('--params', '--params_file', '--params_file_path', default=predict_folder+'params.yaml',
                        help="Path to the file that defines the "
                             "hyperparameters.")
    parser.add_argument('--config',default=predict_folder+'config.yaml', help="Path to the configuration file.")
    parser.add_argument('--runs', type=int, default="1",
                        help="Times to run the inference process.")
    parser.add_argument('--rows', type=int, default=5,
                        help="Number of images per row to be generated.")
    parser.add_argument('--columns', type=int, default=5,
                        help="Number of images per column to be generated.")
    parser.add_argument('--lower', type=float, default=-2,
                        help="Lower bound of the truncated normal random "
                             "variables.")
    parser.add_argument('--upper', type=float, default=2,
                        help="Upper bound of the truncated normal random "
                             "variables.")
    parser.add_argument('--gpu', '--gpu_device_num', type=str, default="0",
                        help="The GPU device number to use.")
    #args = parser.parse_args()
    # colab改成沒有輸入params，等等用指定的方式指定args
    args = parser.parse_args(args=[])
    return args

In [32]:
def setup(folderName):
    """Parse command line arguments, load model parameters, load configurations
    and setup environment."""
    # Parse the command line arguments
    args = parse_arguments(folderName)

    # Load parameters
    # colab改成沒有輸入params，等等用指定的方式指定args
    params = load_yaml(args.params)

    # Load training configurations
    config = load_yaml(args.config)
    update_not_none(config, vars(args))

    # Set unspecified schedule steps to default values
    for target in (config['learning_rate_schedule'], config['slope_schedule']):
        if target['start'] is None:
            target['start'] = 0
        if target['end'] is None:
            target['end'] = config['steps']

    # Make sure result directory exists
    make_sure_path_exists(config['result_dir'])

    # Setup GPUs
    os.environ["CUDA_VISIBLE_DEVICES"] = config['gpu']

    return params, config

In [33]:
def inference_main(folderName):
    """Main function."""
    # Setup
    logging.basicConfig(level=LOGLEVEL, format=LOG_FORMAT)
    params, config = setup(folderName)
    LOGGER.info("Using parameters:\n%s", pformat(params))
    LOGGER.info("Using configurations:\n%s", pformat(config))

    # ============================== Placeholders ==============================
    placeholder_x = tf.placeholder(
        tf.float32, shape=([None] + params['data_shape']))
    placeholder_z = tf.placeholder(
        tf.float32, shape=(None, params['latent_dim']))
    placeholder_c = tf.placeholder(
        tf.float32, shape=([None] + params['data_shape'][:-1] + [1]))
    placeholder_suffix = tf.placeholder(tf.string)

    # ================================= Model ==================================
    # Create sampler configurations
    sampler_config = {
        'result_dir': config['result_dir'],
        'image_grid': (config['rows'], config['columns']),
        'suffix': placeholder_suffix, 'midi': config['midi'],
        'colormap': np.array(config['colormap']).T,
        'collect_save_arrays_op': config['save_array_samples'],
        'collect_save_images_op': config['save_image_samples'],
        'collect_save_pianorolls_op': config['save_pianoroll_samples']}

    # Build model
    model = Model(params)
    if params.get('is_accompaniment'):
        _ = model(
            x=placeholder_x, c=placeholder_c, z=placeholder_z, mode='train',
            params=params, config=config)
        predict_nodes = model(
            c=placeholder_c, z=placeholder_z, mode='predict', params=params,
            config=sampler_config)
    else:
        _ = model(
            x=placeholder_x, z=placeholder_z, mode='train', params=params,
            config=config)
        predict_nodes = model(
            z=placeholder_z, mode='predict', params=params,
            config=sampler_config)

    # Get sampler op
    sampler_op = tf.group([
        predict_nodes[key] for key in (
            'save_arrays_op', 'save_images_op', 'save_pianorolls_op')
        if key in predict_nodes])

    # ================================== Data ==================================
    if params.get('is_accompaniment'):
        data = load_data(config['data_source'], config['data_filename'])
        print('predicted_data_issssss: ',data.shape)

    # ========================== Session Preparation ===========================
    # Get tensorflow session config
    tf_config = tf.ConfigProto()
    tf_config.gpu_options.allow_growth = True

    # Create saver to restore variables
    saver = tf.train.Saver()

    # =========================== Tensorflow Session ===========================
    with tf.Session(config=tf_config) as sess:

        # Restore the latest checkpoint
        LOGGER.info("Restoring the latest checkpoint.")
        with open(os.path.join(config['checkpoint_dir'], 'checkpoint')) as f:
            checkpoint_name = os.path.basename(
                f.readline().split()[1].strip('"'))
        checkpoint_path = os.path.realpath(
            os.path.join(config['checkpoint_dir'], checkpoint_name))
        print('checkpoint_path is:::::',checkpoint_path)
        saver.restore(sess, checkpoint_path)

        # Run sampler op
        for i in range(config['runs']):
            feed_dict_sampler = {
                placeholder_z: scipy.stats.truncnorm.rvs(
                    config['lower'], config['upper'], size=(
                        (config['rows'] * config['columns']),
                        params['latent_dim'])),
                placeholder_suffix: str(i)}  
            if params.get('is_accompaniment'):
                # sample_x = get_samples(
                #     (config['rows'] * config['columns']), data,
                #     use_random_transpose=config['use_random_transpose'])
                # feed_dict_sampler[placeholder_c] = np.expand_dims(
                #     sample_x[..., params['condition_track_idx']], -1)
                feed_dict_sampler[placeholder_c]=data
                # code in "if(i<1)" all are added py paul
                if(i<1):
                    tmp = feed_dict_sampler[placeholder_c]
                    print(i,tmp.shape)
                    for j in range(24):  
                        print(feed_dict_sampler[placeholder_c].shape)  
                        feed_dict_sampler[placeholder_c]=np.vstack([feed_dict_sampler[placeholder_c],tmp])                 
                    	
                    #print('before:\n',sample_x.shape,'\nAfter:\n', sample_x[..., params['condition_track_idx']].shape)
                    #print('params[\'condition_track_idx\']:',params['condition_track_idx'])
                    print('feed_dict_sampler:\n',feed_dict_sampler[placeholder_c].shape)
            sess.run(sampler_op, feed_dict=feed_dict_sampler)


In [34]:
from pypianoroll import Multitrack

def get_midi(route):
  midi_file = Multitrack(route+'fake_x_bernoulli_sampling_0.npz')
  midi_file.write(route+'fake_x_0.mid')
  return midi_file

In [35]:

def predict_trackCondition():
  folderName = 'exp/accompaniment/piano/'
  inference_main(folderName)

def predict_Normal(genre):

  folderName = 'exp/'+genre+'/'
  inference_main(folderName)


In [36]:
def get_genre(genre):
  if(genre is 0):
    return 'normal_pop'
  if(genre is 1):
    return 'default_2'
  if(genre is 2):
    return 'normal_electronic'
  if(genre is 3):
    return 'normal_origin'

# API 拿midi檔
### flask & ngrok source : https://aishuafei.com/google-colab-flask/
### midi:https://stackoverflow.com/questions/28121776/how-do-i-allow-users-to-download-a-midi-file-with-flask-without-getting-a-0-byte



In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask,send_file,request
app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run

@app.route("/trackCondition")
def trackCondition():
  base='/content/drive/MyDrive/museGan/exp/accompaniment/piano/pianorolls/fake_x_bernoulli_sampling/'
  predict_trackCondition()
  midi = get_midi(base)
  new_file = open(base+'fake_x_0.mid', 'rb')
  return send_file(new_file, mimetype='audio/midi')
  
@app.route("/random")
def ramdom():
  # base param
  base ='/content/drive/MyDrive/museGan/exp/'
  lastParam = '/pianorolls/fake_x_bernoulli_sampling/'

  #request data
  rJson = request.json
  genre_num = rJson['genre']#0,1,2
  
  #print('\ngenre_num : ',genre_num,'\n')
  
  genre = get_genre(genre_num)
  
  predict_Normal(genre)
  midi = get_midi(base+genre+lastParam)
  new_file = open(base+genre+lastParam+'fake_x_0.mid', 'rb')
  return send_file(new_file, mimetype='audio/midi')
  
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
werkzeug             INFO      * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://aba2-35-236-131-212.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


musegan.inference    INFO     Using parameters:
{'beat_resolution': 12,
 'condition_track_idx': None,
 'data_shape': [4, 48, 84, 5],
 'is_accompaniment': False,
 'is_conditional': False,
 'latent_dim': 128,
 'nets': {'discriminator': 'default', 'generator': 'default'},
 'use_binary_neurons': False}
musegan.inference    INFO     Using configurations:
{'adam': {'beta1': 0.5, 'beta2': 0.9},
 'batch_size': 64,
 'checkpoint_dir': 'exp/normal_electronic/model/',
 'colormap': [[1.0, 0.0, 0.0],
              [1.0, 0.5, 0.0],
              [0.0, 1.0, 0.0],
              [0.0, 0.0, 1.0],
              [0.0, 0.5, 1.0]],
 'columns': 5,
 'config': 'exp/normal_electronic/config.yaml',
 'data_filename': 'train_x_lpd_5_phr',
 'data_root': None,
 'data_source': 'sa',
 'evaluate_steps': 100,
 'gan_loss_type': 'wasserstein',
 'gpu': '0',
 'initial_learning_rate': 0.001,
 'learning_rate_schedule': {'end': 50000, 'end_value': 0.0, 'start': 45000},
 'log_loss_steps': 100,
 'lower': -2,
 'midi': {'is_drums':

Shape of true data:  (?, 4, 48, 84, 5)
Shape of fake data:  (?, 4, 48, 84, 5)


...

tensorflow           INFO     Restoring parameters from /content/drive/My Drive/museGan/exp/normal_electronic/model/model.ckpt-60000
127.0.0.1 - - [02/Oct/2021 09:26:30] "GET /random HTTP/1.1" 200 -
werkzeug             INFO     127.0.0.1 - - [02/Oct/2021 09:26:30] "GET /random HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2021 09:27:10] "GET / HTTP/1.1" 404 -
werkzeug             INFO     127.0.0.1 - - [02/Oct/2021 09:27:10] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [02/Oct/2021 09:27:10] "GET /favicon.ico HTTP/1.1" 404 -
werkzeug             INFO     127.0.0.1 - - [02/Oct/2021 09:27:10] "GET /favicon.ico HTTP/1.1" 404 -
musegan.inference    INFO     Using parameters:
{'beat_resolution': 12,
 'condition_track_idx': None,
 'data_shape': [4, 48, 84, 5],
 'is_accompaniment': False,
 'is_conditional': False,
 'latent_dim': 128,
 'nets': {'discriminator': 'default', 'generator': 'default'},
 'use_binary_neurons': False}
musegan.inference    INFO     Using configurations:
{'adam': {'beta1': 0.5, 'beta2'

Shape of true data:  (?, 4, 48, 84, 5)
Shape of fake data:  (?, 4, 48, 84, 5)


model                INFO     Building training ops.
model                INFO     Building summaries.
model                INFO     Building prediction nodes.
musegan.inference    INFO     Restoring the latest checkpoint.


checkpoint_path is::::: /content/drive/MyDrive/museGan/exp/default_2/model/model.ckpt-240000
INFO:tensorflow:Restoring parameters from /content/drive/MyDrive/museGan/exp/default_2/model/model.ckpt-240000


tensorflow           INFO     Restoring parameters from /content/drive/MyDrive/museGan/exp/default_2/model/model.ckpt-240000
127.0.0.1 - - [02/Oct/2021 09:27:49] "GET /random HTTP/1.1" 200 -
werkzeug             INFO     127.0.0.1 - - [02/Oct/2021 09:27:49] "GET /random HTTP/1.1" 200 -
